In [72]:
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import svm

In [73]:
feat_names = []
for i in range(334):
    feat_names.append('feat' + str(i + 1))

df_train = pd.read_csv('train10000.csv', names=feat_names)
df_train_targets = pd.read_csv('train10000_Label.csv', names=['targets'])

df_test = pd.read_csv('test10000.csv', names=feat_names)
df_test_targets = pd.read_csv('test10000_label.csv', names=['targets'])

In [74]:
df_train = df_train.join(df_train_targets['targets'])
df_train.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,feat51,feat52,feat53,feat54,feat55,feat56,feat57,feat58,feat59,feat60,feat61,feat62,feat63,feat64,feat65,feat66,feat67,feat68,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat85,feat86,feat87,feat88,feat89,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99,feat100,feat101,feat102,feat103,feat104,feat105,feat106,feat107,feat108,feat109,feat110,feat111,feat112,feat113,feat114,feat115,feat116,feat117,feat118,feat119,feat120,feat121,feat122,feat123,feat124,feat125,feat126,feat127,feat128,feat129,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139,feat140,feat141,feat142,feat143,feat144,feat145,feat146,feat147,feat148,feat149,feat150,feat151,feat152,feat153,feat154,feat155,feat156,feat157,feat158,feat159,feat160,feat161,feat162,feat163,feat164,feat165,feat166,feat167,feat168,feat169,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179,feat180,feat181,feat182,feat183,feat184,feat185,feat186,feat187,feat188,feat189,feat190,feat191,feat192,feat193,feat194,feat195,feat196,feat197,feat198,feat199,feat200,feat201,feat202,feat203,feat204,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219,feat220,feat221,feat222,feat223,feat224,feat225,feat226,feat227,feat228,feat229,feat230,feat231,feat232,feat233,feat234,feat235,feat236,feat237,feat238,feat239,feat240,feat241,feat242,feat243,feat244,feat245,feat246,feat247,feat248,feat249,feat250,feat251,feat252,feat253,feat254,feat255,feat256,feat257,feat258,feat259,feat260,feat261,feat262,feat263,feat264,feat265,feat266,feat267,feat268,feat269,feat270,feat271,feat272,feat273,feat274,feat275,feat276,feat277,feat278,feat279,feat280,feat281,feat282,feat283,feat284,feat285,feat286,feat287,feat288,feat289,feat290,feat291,feat292,feat293,feat294,feat295,feat296,feat297,feat298,feat299,feat300,feat301,feat302,feat303,feat304,feat305,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315,feat316,feat317,feat318,feat319,feat320,feat321,feat322,feat323,feat324,feat325,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,999000.0,46,2.0,46,196000.0,2.0,999000.0,4.0,3.0,999000.0,45,1,999000.0,48,999000.0,40,476,508,375,3,552,300,483,250,982,4,57,5445.0,1.0,17,32,806,8,2,9,20,13,1969,1969,380,363,442,468,372,3,484,454,494,553,468,263,180,406,266,22,56,22,12,1969,3054,4370,11077.0,25,15,36,6,0,11,39,24,6,9,3,0,71,12,13,0,8,9,44,4,17,10,0,0,0,2,1,51,0,0,3,1,4,4,0,0,0,2,0,0,0,10,0,0,0,0,0,10,21,1,8,1,5,8,5,0,75,10,77,4,1,5,9,0,5,6,4,10,1,30,0,4,3,13,3,2,96,2,22,2,3,23,4,1,3,11,13,22,28,12,6,4,3,50,0,64,5,65,0,10,4,1,0,1,0,0,2,35,5,15,22,19,12,22,99,99,94,26,11,70,3,1,2,14,14,20,23,9,14,49,1,34,13,29,95,67,21,1,43,8,0,50,29,9,3,1,86,51,2,13,97,1,5,10,28,12,2,50,0,71,2,76,12,13,59,58,27,118,0,8,83,36,24,0,92,17,10,18,32,23,0,99,11,16,24,17,6,12,6,5,10,10,26,27,13,14,34,41,14,32,26,4,2,0,0,0,51,49,34,19,11,2,78,8,84,16,2,62,0,38,8,71,20,12,1,0,58,9,6,1,7,82,61,14,22,8,64,16,5,18,17,11,7,76,52,1,9,37,3,63,0,0,0,0,0,0,0,8,7,31,7,99,59,19,99,21,6,8,11,27,254,110,1
1,1.0,50,1.0,50,196000.0,1.0,4.0,7.0,5.0,1.0,35,4,40.0,46,1.0,170,504,533,421,3,455,280,385,220,671,4,54,1513.0,7.0,28,47,623,7,3,16,15,11,1952,1954,436,404,480,505,421,3,387,327,374,476,346,242,205,0,149,20,55,20,22,1963,3107,4947,11155.0,33,23,35,7,1,20,32,16,18,5,2,0,59,17,19,10,9,8,46,3,18,12,0,0,0,4,1,14,0,2,12,15,0,7,0,0,1,1,0,1,0,25,0,5,2,10,3,10,40,1,10,1,8,3,4,0,72,15,80,2,2,5,0,1,6,6,6,14,4,13,0,5,3,16,5,4,97,4,15,1,3,17,3,1,1,17,17,20,21,9,12,11,0,53,0,50,7,5

In [75]:
df_test = df_test.join(df_test_targets['targets'])
df_test.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,feat51,feat52,feat53,feat54,feat55,feat56,feat57,feat58,feat59,feat60,feat61,feat62,feat63,feat64,feat65,feat66,feat67,feat68,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat85,feat86,feat87,feat88,feat89,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99,feat100,feat101,feat102,feat103,feat104,feat105,feat106,feat107,feat108,feat109,feat110,feat111,feat112,feat113,feat114,feat115,feat116,feat117,feat118,feat119,feat120,feat121,feat122,feat123,feat124,feat125,feat126,feat127,feat128,feat129,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139,feat140,feat141,feat142,feat143,feat144,feat145,feat146,feat147,feat148,feat149,feat150,feat151,feat152,feat153,feat154,feat155,feat156,feat157,feat158,feat159,feat160,feat161,feat162,feat163,feat164,feat165,feat166,feat167,feat168,feat169,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179,feat180,feat181,feat182,feat183,feat184,feat185,feat186,feat187,feat188,feat189,feat190,feat191,feat192,feat193,feat194,feat195,feat196,feat197,feat198,feat199,feat200,feat201,feat202,feat203,feat204,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219,feat220,feat221,feat222,feat223,feat224,feat225,feat226,feat227,feat228,feat229,feat230,feat231,feat232,feat233,feat234,feat235,feat236,feat237,feat238,feat239,feat240,feat241,feat242,feat243,feat244,feat245,feat246,feat247,feat248,feat249,feat250,feat251,feat252,feat253,feat254,feat255,feat256,feat257,feat258,feat259,feat260,feat261,feat262,feat263,feat264,feat265,feat266,feat267,feat268,feat269,feat270,feat271,feat272,feat273,feat274,feat275,feat276,feat277,feat278,feat279,feat280,feat281,feat282,feat283,feat284,feat285,feat286,feat287,feat288,feat289,feat290,feat291,feat292,feat293,feat294,feat295,feat296,feat297,feat298,feat299,feat300,feat301,feat302,feat303,feat304,feat305,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315,feat316,feat317,feat318,feat319,feat320,feat321,feat322,feat323,feat324,feat325,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,1.0,36,3.0,36,197200.0,1.0,998880.0,998880.0,7.0,2.0,83,1,998880.0,22,0.0,200,450,472,341,10,1052,320,989,290,1977,4,72,6813.0,0.0,23,15,1507,13,4,5,21,12,1984,1985,374,363,446,461,369,9,861,617,854,979,855,550,287,1088,260,26,75,26,21,1997,3170,3760,10941.0,12,10,35,4,0,10,52,34,9,13,3,0,76,12,13,1,8,36,15,14,24,2,0,0,1,8,1,31,0,1,9,5,3,4,0,0,1,3,0,1,1,39,1,1,0,1,0,5,19,1,5,0,2,1,2,0,87,7,90,0,2,4,1,2,4,8,16,12,3,9,0,4,2,12,5,5,97,5,24,2,7,25,14,8,18,2,3,8,26,19,2,4,0,50,0,68,3,70,0,3,3,2,0,0,0,0,1,17,1,12,31,30,14,9,85,99,98,13,4,84,13,7,16,2,5,11,25,17,3,56,0,45,13,15,98,81,26,1,64,3,1,31,93,50,33,3,99,98,9,1,99,0,2,8,13,8,1,50,0,81,1,82,12,26,70,78,41,150,1,0,67,52,20,0,96,0,13,0,86,0,0,99,9,15,26,28,9,14,5,3,5,4,25,24,11,15,35,41,14,35,30,1,1,0,0,0,5,94,62,33,24,4,89,2,93,7,3,0,99,0,10,77,3,26,1,0,14,1,7,0,2,97,95,90,11,16,48,12,1,10,8,5,4,90,84,1,3,72,3,63,0,0,0,0,0,0,0,4,6,58,8,96,63,9,97,24,3,4,7,14,347,111,0
1,0.0,36,1.0,38,196010.0,1.0,4.0,998880.0,7.0,1.0,60,3,998880.0,21,0.0,220,416,441,314,7,806,330,767,300,1215,4,63,7825.0,4.0,28,9,1105,11,4,4,20,17,1991,1991,329,320,403,423,325,7,702,565,714,791,666,588,480,626,305,28,64,28,25,1993,4709,5657,16821.0,13,10,32,5,2,9,55,35,9,13,5,3,75,18,15,5,11,27,15,18,24,3,0,0,1,7,2,13,0,0,4,2,1,1,0,0,2,1,0,2,2,16,2,3,0,5,1,11,22,1,7,1,2,2,2,0,89,9,85,1,9,8,0,2,4,14,14,12,3,5,0,4,8,12,2,2,74,9,23,2,6,28,13,6,5,4,6,11,33,20,1,5,0,51,1,68,6,70,2,30,4,6,2,0

In [76]:
df_train.shape, df_test.shape

((10000, 335), (10000, 335))

In [77]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_train.isnull().sum()

feat1      0
feat2      0
feat3      0
feat4      0
feat5      0
feat6      0
feat7      0
feat8      0
feat9      0
feat10     0
feat11     0
feat12     0
feat13     0
feat14     0
feat15     0
feat16     0
feat17     0
feat18     0
feat19     0
feat20     0
feat21     0
feat22     0
feat23     0
feat24     0
feat25     0
feat26     0
feat27     0
feat28     0
feat29     0
feat30     0
feat31     0
feat32     0
feat33     0
feat34     0
feat35     0
feat36     0
feat37     0
feat38     0
feat39     0
feat40     0
feat41     0
feat42     0
feat43     0
feat44     0
feat45     0
feat46     0
feat47     0
feat48     0
feat49     0
feat50     0
feat51     0
feat52     0
feat53     0
feat54     0
feat55     0
feat56     0
feat57     0
feat58     0
feat59     0
feat60     0
feat61     0
feat62     0
feat63     0
feat64     0
feat65     0
feat66     0
feat67     0
feat68     0
feat69     0
feat70     0
feat71     0
feat72     0
feat73     0
feat74     0
feat75     0
feat76     0
feat77     0

In [78]:
for i in range(df_train.shape[1] - 1):
    feat_name = 'feat' + str(i + 1)
    print(feat_name + ': ', len(df_train[feat_name].unique()))

feat1:  3
feat2:  41
feat3:  8
feat4:  42
feat5:  10
feat6:  3
feat7:  5
feat8:  10
feat9:  10
feat10:  4
feat11:  171
feat12:  9
feat13:  19
feat14:  134
feat15:  3
feat16:  113
feat17:  303
feat18:  262
feat19:  287
feat20:  23
feat21:  1089
feat22:  33
feat23:  969
feat24:  39
feat25:  2535
feat26:  15
feat27:  67
feat28:  2853
feat29:  11
feat30:  49
feat31:  76
feat32:  2385
feat33:  30
feat34:  11
feat35:  24
feat36:  26
feat37:  22
feat38:  62
feat39:  63
feat40:  360
feat41:  335
feat42:  340
feat43:  311
feat44:  339
feat45:  21
feat46:  957
feat47:  798
feat48:  947
feat49:  998
feat50:  948
feat51:  678
feat52:  513
feat53:  841
feat54:  551
feat55:  34
feat56:  71
feat57:  35
feat58:  48
feat59:  63
feat60:  4711
feat61:  5028
feat62:  5544
feat63:  74
feat64:  100
feat65:  53
feat66:  59
feat67:  21
feat68:  100
feat69:  82
feat70:  68
feat71:  99
feat72:  52
feat73:  36
feat74:  40
feat75:  71
feat76:  35
feat77:  28
feat78:  99
feat79:  18
feat80:  49
feat81:  56
feat82:

In [79]:
y_train = df_train['targets'].values
X_train = df_train.drop(columns=['targets'])

y_test = df_test['targets'].values
X_test = df_test.drop(columns=['targets'])

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [80]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [81]:
pred_train = clf.predict(X_train)
pred_train

array([0, 0, 0, ..., 0, 0, 0])

In [82]:
for i in pred_train:
    if i == 1:
        print(i)

1
1
1
1
1
1
1
1
1
1


In [83]:
roc_auc_score(y_train, pred_train)

0.5048954955994995